### Variablen

- Veranstaltungen (Weihnachtsmarkt, Dom (Freimärkte), Fußballspiele, Sonstige) -> Luisa

In [2]:
from bs4 import BeautifulSoup as bs
import requests
import csv
import re

Erstellung des csv.files "veranstaltung_kiel" mit Veranstaltungen aus jahrmarktnord.de

In [11]:
import requests
from bs4 import BeautifulSoup
import csv
import re
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

def fetch_and_extract_data():
    # URL der Webseite
    base_url = 'https://jahrmarktnord.de/category/archiv'  # Standard-Archiv-URL
    year_2019_url = 'https://jahrmarktnord.de/2019/'  # Spezielle URL für 2019
    event_data = []

    # Abruf der 2019-Daten
    print(f"Durchsuche 2019-Seite: {year_2019_url}")
    response = requests.get(year_2019_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        extract_events_from_page(soup, event_data)
    else:
        print(f"Fehler beim Abrufen der 2019-Seite. Status Code: {response.status_code}")

    # Abruf der archivierten Jahre mit paralleler Verarbeitung (für alle 83 Seiten)
    max_pages = 83  # 83 Seiten durchsuchen
    print(f"Durchsuche archivierte Seiten bis Seite {max_pages}...")

    # Paralleles Abrufen der Seiten
    with ThreadPoolExecutor() as executor:
        urls = [f"{base_url}/page/{i}/" for i in range(1, max_pages + 1)]
        responses = list(executor.map(fetch_page, urls))
    
    # Extrahieren der Daten von allen Seiten
    for response in responses:
        if response:
            soup = BeautifulSoup(response.text, 'html.parser')
            extract_events_from_page(soup, event_data)

    # Daten in CSV speichern
    if event_data:
        write_to_csv(event_data)
    else:
        print("Keine relevanten Veranstaltungen gefunden.")

def fetch_page(url):
    """
    Abrufen einer Seite. Rückgabe der Response bei Erfolg, sonst None.
    """
    try:
        print(f"Abrufen von {url}")
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            return response
        else:
            print(f"Fehler beim Abrufen von {url}. Status Code: {response.status_code}")
    except requests.RequestException as e:
        print(f"Request-Fehler bei {url}: {e}")
    return None

def extract_events_from_page(soup, event_data):
    """
    Extrahiert Veranstaltungsdaten aus einer BeautifulSoup-Seite und fügt sie der event_data-Liste hinzu,
    aber nur, wenn "Kiel" im Titel vorkommt.
    """
    events = soup.find_all('article')
    for event in events:
        title_tag = event.find('h3', class_='entry-title')
        if title_tag:
            title = title_tag.text.strip()
            if "Kiel" in title:
                excerpt_tag = event.find('div', class_='mh-excerpt')
                if excerpt_tag:
                    excerpt_text = excerpt_tag.text.strip()
                    date_match = re.search(r'(\d{2}\.\d{2})(?:-(\d{2}\.\d{2})(?:\.(\d{4}))?)?', excerpt_text)
                    if date_match:
                        start_date_raw = date_match.group(1)
                        end_date_raw = date_match.group(2)
                        year = date_match.group(3) or extract_year_from_title(title) or str(datetime.now().year)
                        start_date = parse_date(start_date_raw, year)
                        end_date = parse_date(end_date_raw or start_date_raw, year)
                        event_data.append([title, start_date, end_date])

def extract_year_from_title(title):
    match = re.search(r'(\d{4})', title)
    if match:
        return match.group(1)
    return None

def parse_date(date_str, year):
    try:
        if '.' in date_str:
            date_obj = datetime.strptime(f"{date_str}.{year}", "%d.%m.%Y")
            return date_obj.strftime("%Y-%m-%d")
    except ValueError as e:
        print(f"Fehler beim Parsen des Datums '{date_str}': {e}")
    return None

def write_to_csv(data):
    with open('veranstaltungen_kiel_.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Event Name', 'Start Date (YYYY-MM-DD)', 'End Date (YYYY-MM-DD)'])
        writer.writerows(data)
        print("Daten wurden erfolgreich in die CSV-Datei geschrieben!")

# Aufruf der Funktion
fetch_and_extract_data()


Durchsuche 2019-Seite: https://jahrmarktnord.de/2019/
Durchsuche archivierte Seiten bis Seite 83...
Abrufen von https://jahrmarktnord.de/category/archiv/page/1/
Abrufen von https://jahrmarktnord.de/category/archiv/page/2/
Abrufen von https://jahrmarktnord.de/category/archiv/page/3/
Abrufen von https://jahrmarktnord.de/category/archiv/page/4/
Abrufen von https://jahrmarktnord.de/category/archiv/page/5/
Abrufen von https://jahrmarktnord.de/category/archiv/page/6/
Abrufen von https://jahrmarktnord.de/category/archiv/page/7/
Abrufen von https://jahrmarktnord.de/category/archiv/page/8/
Abrufen von https://jahrmarktnord.de/category/archiv/page/9/
Abrufen von https://jahrmarktnord.de/category/archiv/page/10/
Abrufen von https://jahrmarktnord.de/category/archiv/page/11/
Abrufen von https://jahrmarktnord.de/category/archiv/page/12/
Abrufen von https://jahrmarktnord.de/category/archiv/page/13/
Abrufen von https://jahrmarktnord.de/category/archiv/page/14/
Abrufen von https://jahrmarktnord.de/cate

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import re
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor

def fetch_and_extract_data():
    base_url = 'https://jahrmarktnord.de/category/archiv'  # Archive URL
    year_2019_url = 'https://jahrmarktnord.de/2019/'  # Special URL for 2019
    event_data = []

    # Fetch 2019 data
    print(f"Fetching 2019 page: {year_2019_url}")
    response = requests.get(year_2019_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        extract_events_from_page(soup, event_data)
    else:
        print(f"Error fetching 2019 page. Status Code: {response.status_code}")

    # Fetch archive years
    max_pages = 10
    print(f"Fetching archive pages up to page {max_pages}...")

    with ThreadPoolExecutor() as executor:
        urls = [f"{base_url}/page/{i}/" for i in range(1, max_pages + 1)]
        responses = list(executor.map(fetch_page, urls))
    
    for response in responses:
        if response:
            soup = BeautifulSoup(response.text, 'html.parser')
            extract_events_from_page(soup, event_data)

    # Write to CSV
    if event_data:
        write_to_csv(event_data)
    else:
        print("No relevant events found.")

def fetch_page(url):
    """
    Fetch a single page. Return the response if successful, otherwise None.
    """
    try:
        print(f"Fetching {url}")
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            return response
        else:
            print(f"Error fetching {url}. Status Code: {response.status_code}")
    except requests.RequestException as e:
        print(f"Request error on {url}: {e}")
    return None

def extract_events_from_page(soup, event_data):
    """
    Extracts event data from a BeautifulSoup page and appends to event_data.
    """
    events = soup.find_all('article')
    for event in events:
        title_tag = event.find('h3', class_='entry-title')
        if title_tag:
            title = title_tag.text.strip()
            is_kiel_event = "Kiel" in title and "Kieler Umschlag" not in title
            is_kieler_umschlag = "Kieler Umschlag" in title
            if is_kiel_event or is_kieler_umschlag:
                excerpt_tag = event.find('div', class_='mh-excerpt')
                if excerpt_tag:
                    excerpt_text = excerpt_tag.text.strip()
                    date_match = re.search(r'(\d{2}\.\d{2})(?:-(\d{2}\.\d{2})(?:\.(\d{4}))?)?', excerpt_text)
                    if date_match:
                        start_date_raw = date_match.group(1)
                        end_date_raw = date_match.group(2)
                        year = date_match.group(3) or extract_year_from_title(title) or str(datetime.now().year)
                        start_date = parse_date(start_date_raw, year)
                        end_date = parse_date(end_date_raw or start_date_raw, year)
                        add_event_dates(event_data, start_date, end_date, is_kiel_event, is_kieler_umschlag)

def extract_year_from_title(title):
    """
    Extracts the year from the event title, if available.
    """
    match = re.search(r'(\d{4})', title)
    if match:
        return match.group(1)
    return None

def parse_date(date_str, year):
    """
    Converts a date string (dd.mm or dd.mm.yyyy) to 'YYYY-MM-DD'.
    """
    try:
        if '.' in date_str:
            date_obj = datetime.strptime(f"{date_str}.{year}", "%d.%m.%Y")
            return date_obj.strftime("%Y-%m-%d")
    except ValueError as e:
        print(f"Error parsing date '{date_str}': {e}")
    return None

def add_event_dates(event_data, start_date, end_date, is_kiel_event, is_kieler_umschlag):
    """
    Adds all days of the event to the event_data list, including flags for 'Kiel' and 'Kieler Umschlag'.
    """
    if start_date and end_date:
        current_date = datetime.strptime(start_date, "%Y-%m-%d")
        end_date_obj = datetime.strptime(end_date, "%Y-%m-%d")
        while current_date <= end_date_obj:
            event_data.append([
                current_date.strftime("%Y-%m-%d"),
                1 if is_kiel_event and not is_kieler_umschlag else 0,
                1 if is_kieler_umschlag else 0
            ])
            current_date += timedelta(days=1)

def write_to_csv(data):
    """
    Writes event data to a CSV file.
    """
    with open('veranstaltungen_Kiel_formatiert.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Date (YYYY-MM-DD)', 'Kiel Event (1 if true)', 'Kieler Umschlag (1 if true)'])
        writer.writerows(data)
        print("Data successfully written to 'veranstaltungen_Kiel_formatiert.csv'!")

# Call the function
fetch_and_extract_data()


Fetching 2019 page: https://jahrmarktnord.de/2019/
Fetching archive pages up to page 10...
Fetching https://jahrmarktnord.de/category/archiv/page/1/
Fetching https://jahrmarktnord.de/category/archiv/page/2/
Fetching https://jahrmarktnord.de/category/archiv/page/3/
Fetching https://jahrmarktnord.de/category/archiv/page/4/
Fetching https://jahrmarktnord.de/category/archiv/page/5/
Fetching https://jahrmarktnord.de/category/archiv/page/6/
Fetching https://jahrmarktnord.de/category/archiv/page/7/
Fetching https://jahrmarktnord.de/category/archiv/page/8/
Fetching https://jahrmarktnord.de/category/archiv/page/9/
Fetching https://jahrmarktnord.de/category/archiv/page/10/
Request error on https://jahrmarktnord.de/category/archiv/page/4/: HTTPSConnectionPool(host='jahrmarktnord.de', port=443): Max retries exceeded with url: /category/archiv/page/4/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7e6db10996a0>, 'Connection to jahrmarktnord.de timed out. (connect tim

In [5]:
import requests
from bs4 import BeautifulSoup
import csv
import re
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor

def fetch_and_extract_data():
    # URL der Webseite
    base_url = 'https://jahrmarktnord.de/category/archiv'  # Standard-Archiv-URL
    event_data = []

    # Manuell hinzugefügte Kiel Märkte für 2019
    manual_markets_2019 = [
        ('2019-04-20', '2019-04-28', 1, 0),  # Kiel Frühjahrsmarkt
        ('2019-09-27', '2019-10-06', 1, 0)   # Kiel Herbstmarkt
    ]

    # Hinzufügen der manuellen Markierungen
    for start_date, end_date, kiel_event, kieler_umschlag in manual_markets_2019:
        start = datetime.strptime(start_date, "%Y-%m-%d")
        end = datetime.strptime(end_date, "%Y-%m-%d")
        delta = end - start
        for i in range(delta.days + 1):
            current_date = start + timedelta(days=i)
            event_data.append([current_date.strftime("%Y-%m-%d"), kiel_event, kieler_umschlag])

    # Abruf der archivierten Jahre mit paralleler Verarbeitung (für alle 83 Seiten)
    max_pages = 83  # 83 Seiten durchsuchen
    print(f"Durchsuche archivierte Seiten bis Seite {max_pages}...")

    # Paralleles Abrufen der Seiten
    with ThreadPoolExecutor() as executor:
        urls = [f"{base_url}/page/{i}/" for i in range(1, max_pages + 1)]
        responses = list(executor.map(fetch_page, urls))
    
    # Extrahieren der Daten von allen Seiten
    for response in responses:
        if response:
            soup = BeautifulSoup(response.text, 'html.parser')
            extract_events_from_page(soup, event_data)

    # Daten in CSV speichern
    if event_data:
        write_to_csv(event_data)
    else:
        print("Keine relevanten Veranstaltungen gefunden.")

def fetch_page(url):
    """
    Abrufen einer Seite. Rückgabe der Response bei Erfolg, sonst None.
    """
    try:
        print(f"Abrufen von {url}")
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            return response
        else:
            print(f"Fehler beim Abrufen von {url}. Status Code: {response.status_code}")
    except requests.RequestException as e:
        print(f"Request-Fehler bei {url}: {e}")
    return None

def extract_events_from_page(soup, event_data):
    """
    Extrahiert Veranstaltungsdaten aus einer BeautifulSoup-Seite und fügt sie der event_data-Liste hinzu,
    mit spezifischen Regeln für Kiel-Events.
    """
    events = soup.find_all('article')
    for event in events:
        title_tag = event.find('h3', class_='entry-title')
        if title_tag:
            title = title_tag.text.strip()
            
            # Explicitly exclude "Kieler Woche" events
            if "Kieler Woche" in title:
                continue
            
            # Check if event is in Kiel
            if "Kiel" in title:
                excerpt_tag = event.find('div', class_='mh-excerpt')
                if excerpt_tag:
                    excerpt_text = excerpt_tag.text.strip()
                    date_match = re.search(r'(\d{2}\.\d{2})(?:-(\d{2}\.\d{2})(?:\.(\d{4}))?)?', excerpt_text)
                    if date_match:
                        start_date_raw = date_match.group(1)
                        end_date_raw = date_match.group(2)
                        year = date_match.group(3) or extract_year_from_title(title) or str(datetime.now().year)
                        start_date = parse_date(start_date_raw, year)
                        end_date = parse_date(end_date_raw or start_date_raw, year)

                        # Determine event flags
                        kiel_event = 0  # Default to 0
                        kieler_umschlag = 0  # Default to 0
                        
                        # Special handling for "Kieler Umschlag"
                        if "Kieler Umschlag" in title:
                            kieler_umschlag = 1
                        else:
                            kiel_event = 1

                        # Add each day from the event start to end
                        event_dates = generate_event_dates(start_date, end_date)
                        for event_date in event_dates:
                            event_data.append([event_date, kiel_event, kieler_umschlag])

def extract_year_from_title(title):
    match = re.search(r'(\d{4})', title)
    if match:
        return match.group(1)
    return None

def parse_date(date_str, year):
    try:
        if '.' in date_str:
            date_obj = datetime.strptime(f"{date_str}.{year}", "%d.%m.%Y")
            return date_obj.strftime("%Y-%m-%d")
    except ValueError as e:
        print(f"Fehler beim Parsen des Datums '{date_str}': {e}")
    return None

def generate_event_dates(start_date, end_date):
    """
    Generates a list of dates from the start_date to the end_date.
    """
    dates = []
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    delta = end - start
    for i in range(delta.days + 1):
        current_date = start + timedelta(days=i)
        dates.append(current_date.strftime("%Y-%m-%d"))
    return dates

def write_to_csv(data):
    with open('veranstaltungen_Kiel_formatiert.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Date (YYYY-MM-DD)', 'Kiel Event', 'Kieler Umschlag'])
        writer.writerows(data)
        print("Daten wurden erfolgreich in die CSV-Datei geschrieben!")

# Aufruf der Funktion
fetch_and_extract_data()

Durchsuche archivierte Seiten bis Seite 83...
Abrufen von https://jahrmarktnord.de/category/archiv/page/1/
Abrufen von https://jahrmarktnord.de/category/archiv/page/2/
Abrufen von https://jahrmarktnord.de/category/archiv/page/3/
Abrufen von https://jahrmarktnord.de/category/archiv/page/4/
Abrufen von https://jahrmarktnord.de/category/archiv/page/5/
Abrufen von https://jahrmarktnord.de/category/archiv/page/6/
Abrufen von https://jahrmarktnord.de/category/archiv/page/7/
Abrufen von https://jahrmarktnord.de/category/archiv/page/8/
Abrufen von https://jahrmarktnord.de/category/archiv/page/9/
Abrufen von https://jahrmarktnord.de/category/archiv/page/10/
Abrufen von https://jahrmarktnord.de/category/archiv/page/11/
Abrufen von https://jahrmarktnord.de/category/archiv/page/12/
Abrufen von https://jahrmarktnord.de/category/archiv/page/13/
Abrufen von https://jahrmarktnord.de/category/archiv/page/14/
Abrufen von https://jahrmarktnord.de/category/archiv/page/15/
Abrufen von https://jahrmarktnord